## Métodos de Classificação e Comparação

### Começando com TF-IDF + modelos lineares

**TF-IDF (Term Frequency–Inverse Document Frequency)** é uma forma simples e muito eficaz de transformar texto em números.
A ideia é dar **peso alto** para termos que aparecem com frequência em um ticket (**TF**) e são relativamente **raros no corpus** (**IDF**).

Na prática, usamos TF-IDF com **n-grams** de:
- **palavras** (captura expressões como "password reset"), e
- **caracteres** (ajuda com abreviações, variações e erros de digitação, comuns em tickets).

Isso gera um **vetor esparso de alta dimensão** por ticket. Em seguida, um **classificador linear** aprende um peso para cada feature (n-gram) e decide a classe pela soma ponderada desses pesos.

Por que faz sentido aqui (conforme a análise exploratória):
- **Textos curtos e vocabulário característico** → n-grams capturam bem padrões locais.
- **Dataset grande e desbalanceado** → modelos lineares são fortes e eficientes;

Vamos testar três variantes lineares com TF-IDF e comparar principalmente pelo **F1 macro**.

### Protocolo de avaliação

- Treinamos nos dados de **treino** e avaliamos no **teste**.
- Métrica principal: **F1 macro**, para equilibrar o desempenho entre classes.
- Também reportamos **accuracy** e **F1 weighted** para referência.
- O conjunto de **validação** fica reservado para a avaliação final (quando a solução já estiver definida).
- Mantemos a reprodutibilidade usando `RANDOM_STATE` em splits e modelos.

In [1]:
import pandas as pd
from tqdm.auto import tqdm
from classifier.classifiers import (
    EmbeddingClassifier,
    RagKnnClassifier,
    RagWeightedVoteClassifier,
    TfidfClassifier,
)
from classifier.config import RANDOM_STATE, VALIDATION_SIZE
from classifier.data import load_dataset, train_test_validation_split
from classifier.metrics import evaluate

# Verbosidade do scikit-learn (1+ imprime progresso do solver quando suportado)
SKLEARN_VERBOSE = 1

# Carregar e dividir o dataset para os testes dos classificadores
df, classes = load_dataset()
train_df, test_df, validation_df = train_test_validation_split(
    df,
    validation_size=VALIDATION_SIZE,
    train_size=0.8,
    random_state=RANDOM_STATE,
)

print("Divisão dos dados para os testes dos classificadores:")
print(f"  Treino:      {len(train_df):,} tickets")
print(f"  Teste:       {len(test_df):,} tickets")
print(f"  Validação:   {len(validation_df):,} tickets")

test_texts = test_df["Document"].tolist()
test_labels = test_df["Topic_group"].tolist()

def evaluate_classifier(classifier):
    print(f"\n==> {classifier.name}")
    print("==> Treinando...")
    classifier.fit(train_df)
    print("==> Treinando... OK")

    print(f"==> Classificando no teste: {len(test_texts):,} tickets")
    if isinstance(classifier, (RagKnnClassifier, RagWeightedVoteClassifier)):
        y_pred = classifier.predict(tqdm(test_texts, desc="Predizendo", total=len(test_texts)))
    else:
        y_pred = classifier.predict(test_texts)
    metrics = evaluate(test_labels, y_pred, classes)
    return metrics, y_pred

def print_metrics(metrics, name=None):
    title = f"Métricas ({name})" if name else "Métricas"
    print(title)
    print(f"  accuracy:    {metrics['accuracy']:.4f}")
    print(f"  f1_macro:    {metrics['f1_macro']:.4f}")
    print(f"  f1_weighted: {metrics['f1_weighted']:.4f}")


Divisão dos dados para os testes dos classificadores:
  Treino:      38,109 tickets
  Teste:       9,528 tickets
  Validação:   200 tickets


In [2]:
import random
import numpy as np

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

try:
    import torch

    torch.manual_seed(RANDOM_STATE)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(RANDOM_STATE)
except ImportError:
    pass


### TF-IDF + LinearSVC

O **LinearSVC** é uma SVM linear e costuma ser uma baseline muito forte para **texto em TF-IDF**, porque lida bem com vetores esparsos e de alta dimensão.

In [3]:
tfidf_linear = TfidfClassifier.linear_svc(
    random_state=RANDOM_STATE, verbose=SKLEARN_VERBOSE
)
tfidf_linear_metrics, tfidf_linear_pred = evaluate_classifier(tfidf_linear)
print_metrics(tfidf_linear_metrics, tfidf_linear.name)



==> TF-IDF + LinearSVC
==> Treinando...
[LibLinear].....**
optimization finished, #iter = 57
Objective value = -1525.100119
nSV = 7356
............**.
optimization finished, #iter = 130
Objective value = -840.010881
nSV = 3608
.....*
optimization finished, #iter = 57
Objective value = -2671.115288
nSV = 11603
.....*
optimization finished, #iter = 59
Objective value = -3452.154035
nSV = 15053
.........**
optimization finished, #iter = 98
Objective value = -685.837045
nSV = 3195
......**
optimization finished, #iter = 63
Objective value = -2261.285826
nSV = 9071
.........**
optimization finished, #iter = 99
Objective value = -396.591229
nSV = 2392
...........**
optimization finished, #iter = 119
Objective value = -711.213314
nSV = 3156
==> Treinando... OK
==> Classificando no teste: 9,528 tickets
Métricas (TF-IDF + LinearSVC)
  accuracy:    0.8637
  f1_macro:    0.8641
  f1_weighted: 0.8639


### TF-IDF + Logistic Regression

**Logistic Regression** (multiclasse) é um modelo linear que aprende pesos por feature, mas produz **probabilidades** (via softmax). Em texto com TF-IDF, costuma ser competitiva e ajuda quando queremos interpretar confiança.

Aqui, usamos `class_weight` para mitigar desbalanceamento.

In [4]:
tfidf_logreg = TfidfClassifier.logistic_regression(
    random_state=RANDOM_STATE, verbose=SKLEARN_VERBOSE
)
tfidf_logreg_metrics, tfidf_logreg_pred = evaluate_classifier(tfidf_logreg)
print_metrics(tfidf_logreg_metrics, tfidf_logreg.name)



==> TF-IDF + LogisticRegression
==> Treinando...
Epoch 1, change: 1
Epoch 2, change: 0.21204642
Epoch 3, change: 0.15717247
Epoch 4, change: 0.11930938
Epoch 5, change: 0.19036197
Epoch 6, change: 0.12147558
Epoch 7, change: 0.12233366
Epoch 8, change: 0.12060319
Epoch 9, change: 0.084166522
Epoch 10, change: 0.07419601
Epoch 11, change: 0.087779841
Epoch 12, change: 0.10499883
Epoch 13, change: 0.1285004
Epoch 14, change: 0.0881494
Epoch 15, change: 0.10128045
Epoch 16, change: 0.098006652
Epoch 17, change: 0.13036561
Epoch 18, change: 0.13934241
Epoch 19, change: 0.11259884
Epoch 20, change: 0.075093463
Epoch 21, change: 0.098438402
Epoch 22, change: 0.14155024
Epoch 23, change: 0.10596595
Epoch 24, change: 0.11865774
Epoch 25, change: 0.089767268
Epoch 26, change: 0.15165014
Epoch 27, change: 0.081902808
Epoch 28, change: 0.12846351
Epoch 29, change: 0.086500155
Epoch 30, change: 0.12040105
Epoch 31, change: 0.084020956
Epoch 32, change: 0.094923467
Epoch 33, change: 0.082344761
Ep

/home/gosoares/dev/ticket-classifier/.venv/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



Métricas (TF-IDF + LogisticRegression)
  accuracy:    0.8593
  f1_macro:    0.8605
  f1_weighted: 0.8594


### TF-IDF + SGDClassifier

O **SGDClassifier** treina um modelo linear com *Stochastic Gradient Descent*, sendo eficiente em datasets grandes. Dependendo da loss configurada, ele pode aproximar um SVM (hinge) ou uma regressão logística (log loss).

É uma boa opção quando queremos velocidade e flexibilidade, mantendo a força dos modelos lineares em TF-IDF.

In [5]:
tfidf_sgd = TfidfClassifier.sgd_classifier(
    random_state=RANDOM_STATE, verbose=SKLEARN_VERBOSE
)
tfidf_sgd_metrics, tfidf_sgd_pred = evaluate_classifier(tfidf_sgd)
print_metrics(tfidf_sgd_metrics, tfidf_sgd.name)



==> TF-IDF + SGDClassifier
==> Treinando...
-- Epoch 1
Norm: 12.56, NNZs: 153105, Bias: -1.997780, T: 38109, Avg. loss: 0.128298, Objective: 0.130756
Total training time: 0.05 seconds.
-- Epoch 2
Norm: 3.67, NNZs: 153105, Bias: -2.103788, T: 76218, Avg. loss: 0.117734, Objective: 0.119750
Total training time: 0.09 seconds.
-- Epoch 3
Norm: 3.66, NNZs: 153105, Bias: -2.149386, T: 114327, Avg. loss: 0.116678, Objective: 0.118645
Total training time: 0.12 seconds.
-- Epoch 4
Norm: 8.73, NNZs: 153105, Bias: -2.171614, T: 152436, Avg. loss: 0.116213, Objective: 0.118146
Total training time: 0.16 seconds.
-- Epoch 5
Norm: 9.89, NNZs: 153105, Bias: -2.200554, T: 190545, Avg. loss: 0.115664, Objective: 0.117601
Total training time: 0.19 seconds.
-- Epoch 6
Norm: 4.22, NNZs: 153105, Bias: -2.223582, T: 228654, Avg. loss: 0.115389, Objective: 0.117322
Total training time: 0.22 seconds.
-- Epoch 7
Norm: 9.04, NNZs: 153105, Bias: -2.237853, T: 266763, Avg. loss: 0.115299, Objective: 0.117227
Tota

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    2.4s finished


Métricas (TF-IDF + SGDClassifier)
  accuracy:    0.8546
  f1_macro:    0.8581
  f1_weighted: 0.8551


### Comparação entre TF-IDF

Selecionamos o melhor TF-IDF pelo F1 macro.

In [6]:
tfidf_rows = [
    {
        "method": tfidf_linear.name,
        "accuracy": tfidf_linear_metrics["accuracy"],
        "f1_macro": tfidf_linear_metrics["f1_macro"],
        "f1_weighted": tfidf_linear_metrics["f1_weighted"],
    },
    {
        "method": tfidf_logreg.name,
        "accuracy": tfidf_logreg_metrics["accuracy"],
        "f1_macro": tfidf_logreg_metrics["f1_macro"],
        "f1_weighted": tfidf_logreg_metrics["f1_weighted"],
    },
    {
        "method": tfidf_sgd.name,
        "accuracy": tfidf_sgd_metrics["accuracy"],
        "f1_macro": tfidf_sgd_metrics["f1_macro"],
        "f1_weighted": tfidf_sgd_metrics["f1_weighted"],
    },
]

tfidf_summary = pd.DataFrame(tfidf_rows).sort_values("f1_macro", ascending=False)
tfidf_summary

,method,accuracy,f1_macro,f1_weighted
0,TF-IDF + LinearSVC,0.863665,0.864076,0.863950
1,TF-IDF + LogisticRegression,0.859257,0.860544,0.859427
2,TF-IDF + SGDClassifier,0.854639,0.858083,0.855086


Pelo conjunto de teste, o **TF-IDF + LinearSVC** foi o melhor entre os três modelos lineares (**F1 macro = 0.8641**, **accuracy = 0.8637**, **F1 weighted = 0.8639**), superando Logistic Regression e SGD por uma margem pequena, porém consistente.

Esse resultado faz sentido porque o LinearSVC costuma ser particularmente forte em **vetores TF-IDF esparsos e de alta dimensão**, e o `class_weight="balanced"` ajuda a manter um bom equilíbrio entre classes (o que aparece no F1 macro).

A seguir, vamos manter o **mesmo classificador (LinearSVC)** e trocar apenas a representação de entrada: de TF-IDF para **embeddings**, para avaliar se a informação semântica traz ganho adicional.


### Embeddings + LinearSVC

Em TF-IDF, o modelo depende muito de *tokens/n-grams* específicos. **Embeddings** (sentence-transformers) tentam capturar a **semântica** do texto em um vetor denso.

A ideia aqui é simples: manter o mesmo tipo de classificador linear, mas trocar a representação do texto. Se o dataset tiver muitas variações lexicais para a mesma intenção, embeddings podem ajudar.

In [7]:
embedding_classifier = EmbeddingClassifier.linear_svc(
    random_state=RANDOM_STATE, verbose=SKLEARN_VERBOSE
)
embedding_metrics, embedding_pred = evaluate_classifier(embedding_classifier)
print_metrics(embedding_metrics, embedding_classifier.name)



==> Embeddings + LinearSVC
==> Treinando...


Batches:   0%|          | 0/1191 [00:00<?, ?it/s]

[LibLinear]iter  1 act 2.078e+04 pre 2.078e+04 delta 6.721e-01 f 3.719e+04 |g| 6.184e+04 CG   1
cg reaches trust region boundary
iter  2 act 2.317e+03 pre 2.317e+03 delta 2.449e+00 f 1.641e+04 |g| 3.995e+03 CG   1
cg reaches trust region boundary
iter  3 act 4.844e+03 pre 4.519e+03 delta 3.137e+00 f 1.409e+04 |g| 4.949e+03 CG   2
cg reaches trust region boundary
iter  4 act 2.432e+03 pre 2.058e+03 delta 3.925e+00 f 9.250e+03 |g| 3.092e+03 CG   3
iter  5 act 1.003e+03 pre 8.380e+02 delta 4.560e+00 f 6.818e+03 |g| 2.529e+03 CG   4
cg reaches trust region boundary
iter  6 act 4.488e+02 pre 3.931e+02 delta 5.482e+00 f 5.815e+03 |g| 1.267e+03 CG   5
cg reaches trust region boundary
iter  7 act 1.906e+02 pre 1.813e+02 delta 5.797e+00 f 5.366e+03 |g| 6.000e+02 CG   7
iter  8 act 7.320e+01 pre 7.066e+01 delta 5.797e+00 f 5.176e+03 |g| 2.696e+02 CG  12
iter  9 act 1.132e+01 pre 1.108e+01 delta 5.797e+00 f 5.102e+03 |g| 9.892e+01 CG  14
iter 10 act 1.113e+00 pre 1.110e+00 delta 5.797e+00 f 5.091

Batches:   0%|          | 0/298 [00:00<?, ?it/s]

Métricas (Embeddings + LinearSVC)
  accuracy:    0.7819
  f1_macro:    0.7826
  f1_weighted: 0.7819


Nesta comparação, **embeddings + LinearSVC não ajudou**: o desempenho caiu em relação ao melhor modelo TF-IDF.

- **TF-IDF + LinearSVC:** F1 macro = **0.8641**
- **Embeddings + LinearSVC:** F1 macro = **0.7826**

Uma leitura plausível é que, para este dataset, a **sinalização lexical** (termos e padrões locais) é muito forte para separar classes, e o TF-IDF (word+char n-grams) explora isso diretamente. Já embeddings comprimem a informação em um vetor denso e podem perder parte desses sinais específicos (especialmente se o modelo de embeddings não estiver ajustado ao domínio de tickets).

Com isso, seguimos com **TF-IDF + LinearSVC** como nossa melhor opção de classificador linear supervisionado e, em seguida, avaliamos abordagens baseadas em similaridade (RAG) como referência.


### RAG (k=5): recuperação + classificação

Aqui usamos **RAG apenas como mecanismo de retrieval** (sem LLM): dado um ticket, recuperamos os **K tickets mais similares** do conjunto de treino usando embeddings e similaridade de cosseno.

Em seguida, transformamos esse retrieval em um classificador de duas formas:
- **kNN:** vota pela classe mais frequente entre os K vizinhos.
- **voto ponderado:** soma as similaridades por classe e escolhe a maior soma.

Essas abordagens servem como referência por serem simples, interpretáveis e diretamente ligadas à ideia de “tickets similares”.

#### 3.8.1 Classificação com kNN (k=5)

Neste método, cada ticket do teste é rotulado pela maioria entre os 5 vizinhos mais similares.

In [8]:
rag_knn = RagKnnClassifier(k_similar=5)
rag_knn_metrics, rag_knn_pred = evaluate_classifier(rag_knn)
print_metrics(rag_knn_metrics, rag_knn.name)



==> RAG + kNN (k=5)
==> Treinando...


Batches:   0%|          | 0/1191 [00:00<?, ?it/s]

==> Treinando... OK
==> Classificando no teste: 9,528 tickets


Predizendo:   0%|          | 0/9528 [00:00<?, ?it/s]

Métricas (RAG + kNN (k=5))
  accuracy:    0.8269
  f1_macro:    0.8290
  f1_weighted: 0.8267


#### Classificação com voto ponderado (k=5)

Nesta variação, ao invés de contar votos, somamos os scores de similaridade por classe (um vizinho mais próximo contribui mais).

In [9]:
rag_weighted = RagWeightedVoteClassifier(k_similar=5)
rag_weighted_metrics, rag_weighted_pred = evaluate_classifier(rag_weighted)
print_metrics(rag_weighted_metrics, rag_weighted.name)



==> RAG + Weighted Vote (k=5)
==> Treinando...


Batches:   0%|          | 0/1191 [00:00<?, ?it/s]

==> Treinando... OK
==> Classificando no teste: 9,528 tickets


Predizendo:   0%|          | 0/9528 [00:00<?, ?it/s]

Métricas (RAG + Weighted Vote (k=5))
  accuracy:    0.8269
  f1_macro:    0.8290
  f1_weighted: 0.8267


Nos experimentos acima, os dois classificadores baseados em retrieval (**RAG + kNN** e **RAG + voto ponderado**) tiveram desempenho praticamente idêntico no teste (**F1 macro = 0.8290**, **accuracy = 0.8269**).

Como critério de desempate, faz sentido preferir o **voto ponderado**, pois ele leva em conta a intensidade da similaridade (um vizinho muito próximo contribui mais do que um vizinho distante). Mesmo quando as métricas ficam empatadas, essa regra tende a ser mais estável em casos de fronteira.


### Comparação geral e escolha

Consolidamos os resultados para escolher o melhor método para seguir.

Com a comparação geral, o método eleito para seguir como classificador é o **TF-IDF + LinearSVC** (melhor **F1 macro = 0.8641** no teste).

Resumo do que observamos nesta rodada:
- **TF-IDF + LinearSVC:** F1 macro = **0.8641**
- **RAG (k=5)** (kNN e voto ponderado): F1 macro = **0.8290**
- **Embeddings + LinearSVC:** F1 macro = **0.7826**

Assim, para as próximas fases, seguimos com **TF-IDF + LinearSVC** como classificador supervisionado principal. O RAG continua valioso como mecanismo de **recuperação de tickets similares** para dar contexto (especialmente na etapa de justificativas com LLM).
